In [1183]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import math
import random

In [1184]:
dataset = pd.read_csv('dataset.csv')

In [1185]:
r1 = random.randint(0, 1000)
train_set = dataset.sample(frac=0.8, random_state=r1)
valid_set = dataset.drop(train_set.index)

In [1186]:
X_train=train_set[['F1','F2']]
Y_train=train_set[['T1','T2']]

In [1187]:
Y_bin=Y_train.iloc[:,:1].values
Y_reg=Y_train.iloc[:,1:].values

In [1188]:
for i in range(len(Y_bin)):
  Y_bin[i]-=1

In [1189]:
X=valid_set[['F1','F2']]
Y=valid_set[['T1','T2']]

In [1190]:
a=Y.iloc[:,:1].values
b=Y.iloc[:,1:].values

In [1191]:
for i in range(len(a)):
  a[i]-=1

In [1192]:
input_size = 2
hidden_size1 = 15
hidden_size2 = 15
output_size = 2

In [1193]:
# Initialize weights and biases for the first hidden layer
W1 = np.random.randn(input_size, hidden_size1) 
b1 = np.zeros((1, hidden_size1))

In [1194]:
# Initialize weights and biases for the second hidden layer
W2 = np.random.randn(hidden_size1, hidden_size2) 
b2 = np.zeros((1, hidden_size2))

In [1195]:
# Initialize weights and biases for the output layer
W3 = np.random.randn(hidden_size2, output_size) 
b3 = np.zeros((1, output_size))

In [1196]:
def accuracy(y_pred,y_true):
  accuracy=accuracy_score(y_pred.round(),y_true)
  print(f"Accuracy : {accuracy:.2f}")

In [1197]:
def mse(y_pred,y_true):
  mse = np.mean(np.square(y_true-y_pred))
  print(f"Mean squared error: {mse}")

In [1198]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [1199]:
def tanh(x):
    return np.tanh(x)

In [1200]:
def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [1201]:
def tanh_derivative(z):
    return 1 - np.tanh(z) ** 2

In [1202]:
def binary_cross_entropy(y_pred, y_true):
    eps = 1e-11
    loss = 0
    for i in range(len(y_pred)):
        loss += -(y_true[i] * math.log2(y_pred[i] + eps) + (1 - y_true[i]) * math.log2(1 - y_pred[i] + eps))
    return loss

In [1203]:
def mean_squared_error(y_pred, y_true):
    return np.mean((y_pred - y_true) ** 2)

In [1204]:
def forward(X, W1, W2, W3, b1, b2, b3):
    # First hidden layer
    z1 = np.dot(X, W1) + b1
    a1 = sigmoid(z1)
    
    # Second hidden layer
    z2 = np.dot(a1, W2) + b2
    a2 = tanh(z2)
    
    # Output layer
    z3 = np.dot(a2, W3) + b3
    y1 = sigmoid(z3[:,0])
    y2 = z3[:,1]
    
    return y1, y2, a1, a2

In [1205]:
def backward(X, y1, y2, a1, a2, y_true_1, y_true_2, learning_rate, W1, W2, W3, b1, b2, b3):
    # Output layer
    dL_dy1 = -(y_true_1 - y1) * sigmoid_derivative(y1)
    dL_dy2 = -(y_true_2 - y2) 

    dL_dz3 = np.column_stack((dL_dy1 , dL_dy2))
    
    dW3 = np.dot(a2.T, dL_dz3)
    db3 = np.sum(dL_dz3, axis=0, keepdims=True)
    
    # Second hidden layer
    dL_da2 = np.dot(dL_dz3, W3.T)
    dL_dz2 = dL_da2 * tanh_derivative(a2)

    dW2 = np.dot(a1.T, dL_dz2)
    db2 = np.sum(dL_dz2, axis=0, keepdims=True)
    
    # First hidden layer
    dL_da1 = np.dot(dL_dz2, W2.T)
    dL_dz1 = dL_da1 * sigmoid_derivative(a1)

    dW1 = np.dot(X.T, dL_dz1)
    db1 = np.sum(dL_dz1, axis=0, keepdims=True)
    
    # Update weights and biases
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    
    return W1, b1, W2, b2, W3, b3

In [1206]:
learning_rate = 0.0001
num_epochs = 1000

In [1207]:
for epoch in range(num_epochs):

    for j in range(0, X_train.shape[0],24):
        X_batch=X_train[j:j+24]
        y_true_1=Y_bin[j:j+24]
        y_true_2=Y_reg[j:j+24]

        #Forward Propagation
        y1, y2, a1, a2 = forward(X_batch, W1, W2, W3, b1, b2, b3)
        y1 = y1.reshape((24,1))
        y2 = y2.reshape((24,1))

        #Loss 
        loss_bin = binary_cross_entropy(y1,y_true_1)
        loss_reg = mean_squared_error(y2,y_true_2)
        train_loss = 0.3 * loss_bin + 0.7 * loss_reg
        
        #Backward Propagation and Gradient
        W1, b1, W2, b2, W3, b3 = backward(X_batch, y1, y2, a1, a2, y_true_1, y_true_2, learning_rate, W1, W2, W3, b1, b2, b3) 

    #Validation Loss
    y_val_1,y_val_2,a1,a2=forward(Y, W1, W2, W3, b1, b2, b3)
    loss_bin_val = binary_cross_entropy(y_val_1,a)
    loss_reg_val = mean_squared_error(y_val_2,b)
    loss_val = 0.3 * loss_bin_val + 0.7 * loss_reg_val

    #Print progress
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: \tTraining Loss: {train_loss} \tValidation Loss: {loss_val}") 


Epoch 0: 	Training Loss: [36.87715894] 	Validation Loss: [58.76778732]
Epoch 100: 	Training Loss: [20.04960345] 	Validation Loss: [57.23879775]
Epoch 200: 	Training Loss: [24.86914019] 	Validation Loss: [22.8295257]
Epoch 300: 	Training Loss: [24.79632902] 	Validation Loss: [23.05072805]
Epoch 400: 	Training Loss: [24.78287776] 	Validation Loss: [23.1712905]
Epoch 500: 	Training Loss: [24.7852394] 	Validation Loss: [23.21953898]
Epoch 600: 	Training Loss: [24.78663303] 	Validation Loss: [23.23607301]


<ipython-input-1198-b683d5dd8fa4>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 700: 	Training Loss: [24.78684154] 	Validation Loss: [23.24184605]
Epoch 800: 	Training Loss: [24.78660003] 	Validation Loss: [23.24430612]
Epoch 900: 	Training Loss: [24.78622307] 	Validation Loss: [23.24578315]


In [1208]:
accuracy(y_val_1,a)

Accuracy : 0.80


In [1209]:
mse(y_val_2,b)

Mean squared error: 22.77310104734634
